In [1]:
# Dependencies and set up
import pandas as pd
import numpy as np
import json

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

# City of Chicago -- Crimes - from 2001 to 2021

In [7]:
# Raw crime table
table = pd.read_csv('Crimes_-_2001_to_Present.csv') 
table.head(2)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,-87.6700,"(41.815117282, -87.669999562)",29.0,14924.0,59.0,706.0,3.0,37.0,23.0,108.0
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,-87.7654,"(41.895080471, -87.765400451)",4.0,4299.0,26.0,562.0,45.0,5.0,25.0,67.0


In [5]:
# table.drop((table['Year'] < 2010).index, inplace=True)

In [10]:
table[(table.Year>2010) & (table.Year<2021)]

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,-87.670000,"(41.815117282, -87.669999562)",29.0,14924.0,59.0,706.0,3.0,37.0,23.0,108.0
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,-87.765400,"(41.895080471, -87.765400451)",4.0,4299.0,26.0,562.0,45.0,5.0,25.0,67.0
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,-87.716650,"(41.937405765, -87.716649687)",15.0,21538.0,22.0,216.0,12.0,39.0,7.0,168.0
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,-87.755121,"(41.881903443, -87.755121152)",11.0,22216.0,26.0,696.0,23.0,32.0,25.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7374182,12444566,JE327093,12/01/2020 12:00:00 AM,062XX S WESTERN AVE,0810,THEFT,OVER $500,BANK,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7374192,12444203,JE325895,07/08/2014 09:00:00 AM,082XX S OGLESBY AVE,5002,OTHER OFFENSE,OTHER VEHICLE OFFENSE,STREET,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7374194,12444560,JE327298,11/01/2020 08:00:00 AM,062XX S CORNELL DR,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7374210,12445067,JE327968,04/01/2015 12:00:00 AM,021XX S CALIFORNIA AVE,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,OTHER (SPECIFY),False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
table.reset_index(inplace=True)

In [12]:
table.head()

,index,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats
0,0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,...,-87.670000,"(41.815117282, -87.669999562)",29.0,14924.0,59.0,706.0,3.0,37.0,23.0,108.0
1,1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,...,-87.765400,"(41.895080471, -87.765400451)",4.0,4299.0,26.0,562.0,45.0,5.0,25.0,67.0
2,2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,...,-87.716650,"(41.937405765, -87.716649687)",15.0,21538.0,22.0,216.0,12.0,39.0,7.0,168.0
4,4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,...,-87.755121,"(41.881903443, -87.755121152)",11.0,22216.0,26.0,696.0,23.0,32.0,25.0,81.0


In [ ]:
# Groupby Primary type
# df = table.groupby(['Primary Type']).count()['Description']
# df

In [13]:
# Dropping table
table = table.drop(columns=['Case Number', 'Location', 'Updated On', 'Block', 'Location Description', 'Community Area', 'Beat', 'District', 'Ward', 'FBI Code', 'X Coordinate', 'Y Coordinate' ])
table.head()

,index,ID,Date,IUCR,Primary Type,Description,Arrest,Domestic,Year,Latitude,Longitude,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats
0,0,10224738,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,2015,41.815117,-87.670000,29.0,14924.0,59.0,706.0,3.0,37.0,23.0,108.0
1,1,10224739,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,False,False,2015,41.895080,-87.765400,4.0,4299.0,26.0,562.0,45.0,5.0,25.0,67.0
2,2,11646166,09/01/2018 12:01:00 AM,0810,THEFT,OVER $500,False,True,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,10224740,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),True,False,2015,41.937406,-87.716650,15.0,21538.0,22.0,216.0,12.0,39.0,7.0,168.0
4,4,10224741,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,False,True,2015,41.881903,-87.755121,11.0,22216.0,26.0,696.0,23.0,32.0,25.0,81.0


In [14]:
# Dropping NAN from latitude
table = table[table['Latitude'].notna()]
table.head()

,index,ID,Date,IUCR,Primary Type,Description,Arrest,Domestic,Year,Latitude,Longitude,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats
0,0,10224738,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,2015,41.815117,-87.670000,29.0,14924.0,59.0,706.0,3.0,37.0,23.0,108.0
1,1,10224739,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,False,False,2015,41.895080,-87.765400,4.0,4299.0,26.0,562.0,45.0,5.0,25.0,67.0
3,3,10224740,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),True,False,2015,41.937406,-87.716650,15.0,21538.0,22.0,216.0,12.0,39.0,7.0,168.0
4,4,10224741,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,False,True,2015,41.881903,-87.755121,11.0,22216.0,26.0,696.0,23.0,32.0,25.0,81.0
5,5,10224742,09/05/2015 10:55:00 AM,0610,BURGLARY,FORCIBLE ENTRY,False,False,2015,41.744379,-87.658431,18.0,21554.0,70.0,575.0,13.0,59.0,20.0,237.0


In [15]:
table['Month'] = table['Date'].str[0:2].astype(int)
table.head()

,index,ID,Date,IUCR,Primary Type,Description,Arrest,Domestic,Year,Latitude,Longitude,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats,Month
0,0,10224738,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,2015,41.815117,-87.670000,29.0,14924.0,59.0,706.0,3.0,37.0,23.0,108.0,9
1,1,10224739,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,False,False,2015,41.895080,-87.765400,4.0,4299.0,26.0,562.0,45.0,5.0,25.0,67.0,9
3,3,10224740,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),True,False,2015,41.937406,-87.716650,15.0,21538.0,22.0,216.0,12.0,39.0,7.0,168.0,9
4,4,10224741,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,False,True,2015,41.881903,-87.755121,11.0,22216.0,26.0,696.0,23.0,32.0,25.0,81.0,9
5,5,10224742,09/05/2015 10:55:00 AM,0610,BURGLARY,FORCIBLE ENTRY,False,False,2015,41.744379,-87.658431,18.0,21554.0,70.0,575.0,13.0,59.0,20.0,237.0,9


# Postgres connection - Chicago crime table

In [16]:
rds_connection_string = "postgres:postgres@localhost:5432/Project_03"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
table.to_sql(name='Chicago_crime', con=engine, if_exists='append', index=False)